This notebook is to fine tune the XGBoost model and perform evaulation on user data for 7 day Free Trial.

The whole process has been divided into 2 notebooks: 

- Part 1: Data Preprocessing: 6.0_sk_fine_tuning_FT_propensity_data_preprocessing.ipynb (this notebook)
- part 2: Data Modeling and Evaluation: 6.0_sk_fine_tuning_FT_propensity_data_modeling.ipynb 

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
import os
import json
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

In [16]:
RANDOM_STATE=101
SMALL_DATASET=False

BUCKET = "datascience-hbo-users"
PREFIX = "users/sk/FT_propensity/7_day"

# 0. Snowflake Connection

In [8]:
# Run only when life gives you lemons (or you have issues connecting to snowflake)

# %%bash
# pip install --upgrade pip
# pip3 install snowflake-connector-python
# pip3 install sqlparse
# # pip3 install botocore==1.14.17

# # awscli-compatible botocore
# pip3 install botocore==1.15.39


In [9]:
# %%bash
# # # # # # # # # # 
# # Libs config.
# # # # # # # # # # 

# CFFI_VERSION=$(python -m pip list 2>/dev/null | grep cffi | awk '{print $2}')
# # echo "Current cffi version="$CFFI_VERSION

# pip install --upgrade pip

# if [[ "$CFFI_VERSION" == "1.10.0" ]]
# then
#     echo "Uninstalling cffi"
#     pip uninstall --yes cffi
# fi

# yum_log=$(sudo yum history new && sudo yum install -y libffi-devel openssl-devel)
# pip_log=$(python -m pip install --upgrade pip snowflake-connector-python tabulate ipywidgets sqlparse textile)
# yum_log=$(jupyter nbextension enable --py widgetsnbextension 2> /dev/null)

# if [[ "$CFFI_VERSION" == "cffi (1.10.0)" ]]
# then 
#    echo "Configuration has changed; restart notebook"
# fi

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (20.1.1)


In [10]:
# import json
# import snowflake.connector
# from abc import ABCMeta, abstractmethod

# class Credentials(metaclass=ABCMeta):
#     pass
    
    
# class SSMPSCredentials(Credentials):
#     def __init__(self, secretid: str):
#         self._secretid = secretid
#         self._secrets = {}
        
#     def get_keys(self):
#         """
#         credential fetching 
#         """
#         _aws_sm_args = {'service_name': 'secretsmanager', 'region_name': 'us-east-1'}
#         secrets_client = boto3.client(**_aws_sm_args)
#         get_secret_value_response = secrets_client.get_secret_value(SecretId=self._secretid)
#         return get_secret_value_response
    
    
# class BaseConnector(metaclass=ABCMeta):
#     @abstractmethod
#     def connect(self):
#         raise NotImplementedError
        

# class SnowflakeConnector(BaseConnector):
#     def __init__(self, credentials: Credentials):
#         keys = credentials.get_keys()
#         self._secrets = json.loads(keys.get('SecretString', "{}"))

#     def connect(self, dbname: str, schema: str = 'DEFAULT'):
#         ctx = snowflake.connector.connect(
#             user=self._secrets['login_name'],
#             password=self._secrets['login_password'],
#             account=self._secrets['account'],
#             warehouse=self._secrets['warehouse'],
#             database=dbname,
#             schema=schema
#         )

#         return ctx

# MAX_FETCHED_RESULTSET_SIZE = MAX_QUERY_RETURN_SIZE

# def execute_query(query: str):
#     cur = ctx.cursor()
#     try:
#         cur.execute(query)
#         while True:
#             results = cur.fetchmany(MAX_FETCHED_RESULTSET_SIZE)
#             if len(results) == 0:
#                 break
#             for row in results:
#                 yield row
#     finally:
#         cur.close()
         
# def execute_insert_query(query: str):
#     cur = ctx.cursor()
#     try:
#         cur.execute(query)
#     finally:
#         cur.close()


# connector = SnowflakeConnector(SSMPSCredentials(SF_CREDS))
# ctx = connector.connect(dbname=DB_NAME, schema=SCHEMA)

In [11]:
# //Following query is run on the Snowflake console. Dropping of columns have been moved from SageMaker to Snowflake intentionally to reduce the data preprocessing load from the Sagemaker environment
# USE ROLE MAX_ANALYTICS;
# USE WAREHOUSE MAX_ANALYTICS_USER;
# USE DATABASE MAX_DEV;
# USE SCHEMA WORKSPACE;

# SHOW PARAMETERS LIKE 'WEEK_START%';
# ALTER SESSION SET WEEK_START = 7;

# /****************************
# *   SET INPUT TABLE NAMES   *
# *****************************/

# SET ANALYSIS_START_DATE = '2020-05-27';
# SET REFRESH_DATE = '2020-07-02';
# SET FT_FULLJOIN_TABLE = 'MAX_DEV.WORKSPACE.MAX_FREETRIAL_BASE';
# SET MODEL_INPUT_TABLE ='MAX_DEV.WORKSPACE.SK_FT_7_DAY_MODEL_INPUT_TABLE';

# CREATE OR REPLACE TABLE IDENTIFIER($MODEL_INPUT_TABLE) AS
# SELECT
#     *
# FROM IDENTIFIER($FT_FULLJOIN_TABLE)
# WHERE (UPPER(PROVIDER) IN ('DTC', 'APPLE') AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-8,($REFRESH_DATE)))
#     OR 
#     (UPPER(PROVIDER) IN ('SAMSUNG') AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-11,($REFRESH_DATE )))
#     OR 
#     (UPPER(PROVIDER) IN ('GOOGLE') AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-13,($REFRESH_DATE )))
# ;

# //QA
# SELECT
#     'DESTINATION' AS CAT
#     , PERIOD_RANK
#     , COUNT(*) AS RECORDS
#     , COUNT(DISTINCT HBO_UUID) AS COUNT_UUID
# //    , MIN(FREE_TRIAL_START_DT)
# //    , MAX(FREE_TRIAL_START_DT)
# FROM  IDENTIFIER($MODEL_INPUT_TABLE)
# GROUP BY PERIOD_RANK
# UNION 
# SELECT
#     'SOURCE' AS CAT
#     , PERIOD_RANK
#     , COUNT(*) AS RECORDS
#     , COUNT(DISTINCT HBO_UUID) AS COUNT_UUID
#     , MIN(FREE_TRIAL_START_DT)
#     , MAX(FREE_TRIAL_START_DT)
# FROM IDENTIFIER($FT_FULLJOIN_TABLE)
# WHERE (UPPER(PROVIDER) IN ('DTC', 'APPLE')
#        AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-8,($REFRESH_DATE)))
#     OR 
#     (UPPER(PROVIDER) IN ('SAMSUNG')
#      AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-11,($REFRESH_DATE )))
#     OR 
#     (UPPER(PROVIDER) IN ('GOOGLE')
#      AND FREE_TRIAL_START_DT BETWEEN ($ANALYSIS_START_DATE) AND DATEADD(DAY,-13,($REFRESH_DATE )))
# GROUP BY PERIOD_RANK
# ORDER BY PERIOD_RANK
# ;


# --##################################################################################################################
# ----STEP 2: Drop unnecessary columns
# --##################################################################################################################

# ALTER TABLE IDENTIFIER($MODEL_INPUT_TABLE) 
# DROP COLUMN FREE_TRIAL_START_DT
#             , FREE_TRIAL_END_DT
#             , FIRST_STREAM_DATE_ALL_HIST
#             , BILLING_CYCLE_START_DT
#             , BILLING_CYCLE_END_DT
#             , FIRST_WATCHED_ASSET_CLASS_SUB
#             , TOTAL_HBONOW_WATCH_SEC
#             , ANALYSIS_START_DT
#             , ANALYSIS_END_DT           
#             , FIRST_WATCHED_ASSET_TITLE_ADJ
#             , TOTAL_WATCH_SEC
#             , OUTLIER_DESC
#             , NUM_DAYS_IN_FREE_TRIAL
#             , FIRST_TURN_OFF_AUTORENEW_DATE
#             , LAST_TURN_OFF_AUTORENEW_DATE
#             , FIRST_STREAM_DATE_IN_WINDOW
#             , FIRST_STREAM_DATE_IN_WINDOW_ADJ
#             , LAST_STREAM_DATE_IN_WINDOW
#             , LAST_STREAM_DATE_IN_WINDOW_ADJ
#             , LAST_SESSION_START_TIME
#             , FIRST_WATCHED_ASSET_TITLE
#             , SERIES_VIEWING_CAT_LABEL
# ;
     
# --##################################################################################################################
# ----STEP 3: Copy data from Snowflake to S3
# --##################################################################################################################

# COPY INTO @DATASCIENCE_HBO_USERS/users/sk/FT_propensity/7_day/snowflake-hbomax-staging/raw_data.csv
#      FROM IDENTIFIER($MODEL_INPUT_TABLE) 
#      FILE_FORMAT = (TYPE=CSV FIELD_OPTIONALLY_ENCLOSED_BY = '"' NULL_IF = ('missing') BINARY_FORMAT = utf8 COMPRESSION=NONE)
#      OVERWRITE = TRUE
#  --    SINGLE = TRUE
#      HEADER = TRUE
#      MAX_FILE_SIZE = 5368709120;

In [12]:
# query = f"SELECT * FROM {TABLE_NAME} LIMIT 1000"
# query_columns = f"SHOW COLUMNS IN {TABLE_NAME}"

# df_columns = list(pd.DataFrame(execute_query(query_columns)).iloc[:,2])
# df_raw_sf = pd.DataFrame(execute_query(query),columns = df_columns)

In [16]:
import gc
gc.collect()

1984

## Code for preprocessing data on remote instance

In [1]:
%%writefile preprocessing.py

import argparse
import os
import warnings

import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

print('modules import completed')

def create_unique_index(data):
#     idx_series = data.index.to_series()
#     idx_gb = idx_series.groupby([data.HBO_UUID, data.PERIOD_RANK])
#     idx_tf = idx_gb.transform("first")
#     data["UNIQUE_ID"]=idx_tf
    data["UNIQUE_ID"]=data["HBO_UUID"]+":"+data["PERIOD_RANK"]
    print(f"data head: {data[['UNIQUE_ID', 'HBO_UUID', 'PERIOD_RANK']].head()}")
#     assert data.UNIQUE_ID.nunique()==data.HBO_UUID.nunique()*data.PERIOD_RANK.nunique()
    df_step_1=data.set_index(['UNIQUE_ID'])
    return df_step_1

def fill_missing(data):
    data.FIRST_WATCHED_ASSET_CLASS_SUB_ADJ = data.FIRST_WATCHED_ASSET_CLASS_SUB_ADJ.str.replace("unknown","missing")          
    return data

def transform(data):
    print(f"input data shape: {data.shape}")
    print(f"input data size: {data.memory_usage(index=True).sum()}")
    print(f"original dataframe head: {data.head()}")
    print(f"original dataframe index name: {data.index.name}")
    print(f"original dataframe index 5 values: {data.index[:5]}")
    cols_to_one_hot_encode=["PROVIDER"
                        , "FIRST_WATCHED_ASSET_CLASS_SUB_ADJ"
                        , "FT_SEGMENT", "FT_SUB_SEGMENT"]

    id_list=["UNIQUE_ID", "HBO_UUID"]
    cols_to_exclude_from_num_conversion=cols_to_one_hot_encode+id_list

    obj_to_num_col_list=[col for col in data.select_dtypes(include=['object']).columns if col not in cols_to_exclude_from_num_conversion]

    counter=0
    for col in obj_to_num_col_list:
        counter+=1
        print(f"will process column {col}")
        data[col]=data[col].astype(np.float16)
        print(f"processed column {col}")
        print(f"No. of columns processed: {counter}")
     
    print("size of data after converting object to numeric columns")
    print(f"input data size: {data.memory_usage(index=True).sum()}")
            
    encoder = OneHotEncoder(handle_unknown="ignore", dtype=np.uint8)
    print(f"Columns to onehot encode: {cols_to_one_hot_encode}")
    encode_df=data[cols_to_one_hot_encode]
    print(f"shape of dataframe to be onehot encoded: {encode_df.shape}")
    encoded_mtx=encoder.fit_transform(encode_df).toarray()
    encoded_df=pd.DataFrame(encoded_mtx, index=data.index, columns=encoder.get_feature_names())
    print("size of categorical column data after one hot encoding")
    print(f"encoded_df data size: {encoded_df.memory_usage(index=True).sum()}")
    print(f"shape of categorical dataframe after onehot encoding: {encoded_df.shape}")
    print(f"encoded dataframe head: {encoded_df.head()}")
    print(f"encoded dataframe index name: {encoded_df.index.name}")
    print(f"encoded dataframe index 5 values: {encoded_df.index[:5]}")
    
    del encode_df
    del encoded_mtx

    df_step_1=pd.concat([data, encoded_df], axis=1)
    
    print("size of combined data set with onehot encoded columns")    
    print(f"shape of full dataframe after merging onehot encoded dataframe: {df_step_1.shape}")
    print(f"df_step_1 data size: {df_step_1.memory_usage(index=True).sum()}")
    
    del data
    del encoded_df
    
    # Keeping column "FT_SEGMENT" in the final to join it back while presenting results
    cols_to_drop=[col for col in cols_to_one_hot_encode if col not in "FT_SEGMENT"]
    # If we do not need columns in the final dataframe, we should drop cols_to_one_hot_encode
    df_step_2=df_step_1.drop(cols_to_drop, axis=1)
    del df_step_1
    df_step_2["UNIQUE_ID"]=df_step_2.index
    return df_step_2

# def add_weight_col(data, weight):
#     stream_mask = (data["NUM_STREAMS_ADJ"]>0)
#     dormant_mask = (data["NUM_STREAMS_ADJ"]==0)
#     data.loc[stream_mask, "WEIGHT"] = 1-weight
#     data.loc[dormant_mask, "WEIGHT"] = weight
#     cols=list(data.columns)
#     cols.insert(1, cols.pop(cols.index("WEIGHT")))
#     data = data.loc[:, cols]
#     return data

def split_dataframe(df, strat_column, target, test_size=0.1):
    df_train,df_test = train_test_split (df, test_size = test_size, stratify=df[[strat_column, target]], random_state=101)
    df_train,df_val = train_test_split (df_train, test_size = test_size, stratify=df_train[[strat_column, target]], random_state=101)
    
    print(f"Distribution by PERIOD_RANK and FLG_TARGET")
    
    print(f"train: {df_train.groupby(by=[strat_column, target]).size().reset_index().rename(columns={0:'COUNT_UUID'})}")
    print(f"test: {df_test.groupby(by=[strat_column, target]).size().reset_index().rename(columns={0:'COUNT_UUID'})}")
    print(f"val: {df_val.groupby(by=[strat_column, target]).size().reset_index().rename(columns={0:'COUNT_UUID'})}")
    
    df_train_X,df_train_y = df_train.drop([target],axis=1), df_train[[target]]
    df_test_X,df_test_y = df_test.drop([target],axis=1), df_test[[target]]
    df_val_X,df_val_y = df_val.drop([target],axis=1), df_val[[target]]
    return df_train_X, df_train_y , df_test_X, df_test_y, df_val_X, df_val_y

if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', type=float, default=0.1)
    parser.add_argument('--csv-weight', type=float, default=0.7)
    args, _ = parser.parse_known_args()
    
    # how train-test-split-ratio will pick up value from args.train_test_split_ratio
    split_ratio = args.train_test_split_ratio
    csv_weight = args.csv_weight
    
    input_data_path = os.path.join('/opt/ml/processing', 'input')
    
    print('reading csv files from s3')
    all_files = glob.glob(os.path.join(input_data_path, "*.csv"))
    print(f"all files: {all_files}")
    df = pd.concat((pd.read_csv(f, header=0, encoding="utf-8") for f in all_files))
    
    print(len(df.columns),'\n')
    print(df.columns)
    
    print('read file completed ')
    
    print('transorming input data')
    
    clean_df_step_1=create_unique_index(df)
    clean_df_step_2=fill_missing(clean_df_step_1)
    clean_df_step_3=transform(clean_df_step_2)
#     clean_df_step_4=add_weight_col(clean_df_step_3, csv_weight)
    
    print('transformation completed','\n')
     
    df_train_X, df_train_y, df_test_X, df_test_y, df_val_X, df_val_y=split_dataframe(clean_df_step_3, "PERIOD_RANK", "FLG_TARGET", split_ratio)
    
    print(f"train X shape: {df_train_X.shape}")
    print(f"train y shape: {df_train_y.shape}")
    print(f"test X shape: {df_test_X.shape}")
    print(f"test y shape: {df_test_y.shape}")
    print(f"val X shape: {df_val_X.shape}")
    print(f"val y shape: {df_val_y.shape}")
    
    print(f"data split according to period rank")
    
    print(f"train data: {df_train_X.PERIOD_RANK.value_counts(dropna=False)}")
    print(f"test data: {df_test_X.PERIOD_RANK.value_counts(dropna=False)}")
    print(f"val data: {df_val_X.PERIOD_RANK.value_counts(dropna=False)}")
    
    print(f"Normalized data split according to period rank")
    
    print(f"Normalized train data: {df_train_X.PERIOD_RANK.value_counts(dropna=False, normalize=True)}")
    print(f"Normalized test data: {df_test_X.PERIOD_RANK.value_counts(dropna=False, normalize=True)}")
    print(f"Normalized val data: {df_val_X.PERIOD_RANK.value_counts(dropna=False, normalize=True)}")
    
    print('Extracting side info')
    
    side_info_cols=["UNIQUE_ID", "HBO_UUID", "PERIOD_RANK", "FT_SEGMENT"]
    df_train_side_info=df_train_X[side_info_cols]
    df_test_side_info=df_test_X[side_info_cols]
    df_val_side_info=df_val_X[side_info_cols]
    
    # keep PERIOD_RANK in the train, test and val sets
    cols_to_drop=[col for col in side_info_cols if col!="PERIOD_RANK"]
    df_train_X = df_train_X.drop(cols_to_drop, axis=1)
    df_test_X = df_test_X.drop(cols_to_drop, axis=1)
    df_val_X = df_val_X.drop(cols_to_drop, axis=1)
    
    assert "PERIOD_RANK" in df_train_X.columns
    assert "PERIOD_RANK" in df_test_X.columns
    assert "PERIOD_RANK" in df_val_X.columns
    
    print(f"Side info train X shape: {df_train_side_info.shape}")
    print(f"Side info test X shape: {df_test_side_info.shape}")
    print(f"Side info val X shape: {df_val_side_info.shape}")
    
    print(f"train X shape: {df_train_X.shape}")
    print(f"train y shape: {df_train_y.shape}")
    print(f"test X shape: {df_test_X.shape}")
    print(f"test y shape: {df_test_y.shape}")
    print(f"val X shape: {df_val_X.shape}")
    print(f"val y shape: {df_val_y.shape}")
    
    print("Extracted side info")
    
    print("Extracting column names")
    
    print(f"Number of train columns: {df_train_X.shape[1]}")
    print(f"Number of test columns: {df_test_X.shape[1]}")
    print(f"Number of val columns: {df_val_X.shape[1]}")

    df_train_X.columns.to_series().to_csv("/opt/ml/processing/train/train_columns.csv", header = False, index = False)
    df_test_X.columns.to_series().to_csv("/opt/ml/processing/test/test_columns.csv", header = False, index = False)
    df_val_X.columns.to_series().to_csv("/opt/ml/processing/val/val_columns.csv", header = False, index = False)
        
    print("Extracted column names")
        
    print("writing files back to s3")
    
    # change headers back to False after validation
    pd.concat([df_train_y,df_train_X],axis=1).to_csv("/opt/ml/processing/train/train.csv", header = 0, index = False)
    pd.concat([df_val_y,df_val_X],axis=1).to_csv("/opt/ml/processing/val/val.csv", header = 0, index = False)
    pd.concat([df_test_y,df_test_X],axis=1).to_csv("/opt/ml/processing/test/test.csv", header = 0, index = False)
    
    df_train_side_info.to_csv("/opt/ml/processing/train/train_side_info.csv", header = 0, index = False)
    df_test_side_info.to_csv("/opt/ml/processing/val/test_side_info.csv", header = 0, index = False)
    df_val_side_info.to_csv("/opt/ml/processing/test/val_side_info.csv", header = 0, index = False)
    
    print("processing write to s3")
    
    

Overwriting preprocessing.py


In [2]:
import boto3
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()
sess = sagemaker.Session()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.r5.24xlarge',
                                     output_kms_key='alias/aws/s3',
                                     instance_count=1,
                                     base_job_name='FT-Propensity')

In [3]:
%%time

from sagemaker.processing import ProcessingInput, ProcessingOutput

output_destination = 's3://datascience-hbo-users/users/sk/FT_propensity/7_day/model_input_data'
input_source = 's3://datascience-hbo-users/users/sk/FT_propensity/7_day/snowflake-hbomax-staging'

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(
                        source=input_source,
                        destination='/opt/ml/processing/input')],
                      outputs=[
                               ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/train',
                                                destination=output_destination),
                               
                               ProcessingOutput(output_name='test_data',
                                                source='/opt/ml/processing/test',
                                                destination=output_destination),
                          
                               ProcessingOutput(output_name='val_data',
                                               source='/opt/ml/processing/val',
                                               destination=output_destination)
                               
                               
                              ],
                      arguments=['--train-test-split-ratio', '0.1']
                     )

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  FT-Propensity-2020-07-17-15-29-16-240
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://datascience-hbo-users/users/sk/FT_propensity/7_day/snowflake-hbomax-staging', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-613630599026/FT-Propensity-2020-07-17-15-29-16-240/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'S3Output': {'S3Uri': 's3://datascience-hbo-users/users/sk/FT_propensity/7_day/model_input_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'test_data', 'S3Output': {'S3Uri': 's3://datascience-hbo-users/users/sk/FT_propensity/7_day/model_i

size of categorical column data after one hot encoding
encoded_df data size: 355914468
shape of categorical dataframe after onehot encoding: (6844509, 44)
encoded dataframe head:                                                     x0_apple  ...  x3_missing
UNIQUE_ID                                                     ...            
ed260580ada8b2c465a03f3045b2b520ff4828e60de2ece...         0  ...           0
aa5efcf38f08f88cc7caf097be193c33339d38dc08a3328...         1  ...           0
7bb0fb5e26644ac4107126c3d3eb284a66a8d6eb732cb86...         1  ...           0
b68e5a014015e799c4dd095a764c2c03da3bbe61b1b2b93...         0  ...           0
d7dbd4731833a27edf799b0377a115e847748808c6bb17c...         1  ...           0

[5 rows x 44 columns]
encoded dataframe index name: UNIQUE_ID
encoded dataframe index 5 values: Index(['ed260580ada8b2c465a03f3045b2b520ff4828e60de2eced8324ea4165687cad:0         3\n1         2\n2         2\n3         1\n4         3\n         ..\n124269    1\n124270    1\n1